# Data Engineering

## Creation of base dataset


In [1]:
import sys
import pandas as pd
import numpy as np
import seaborn as sns
sys.path.append(r"../data/")
import scrape_data

In [2]:
try: 
    df = pd.read_csv('C:\Users\priya\dataset\DATA SET\Weather Forecast\Data\Classified_Distdata')
except FileNotFoundError as e:
    scrape_data.create_datasets()
    df = pd.read_csv('C:\Users\priya\dataset\DATA SET\Weather Forecast\Data\Classified_Distdata')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (400683380.py, line 2)

In [3]:
df.date = pd.to_datetime(df.date, format=r"%Y%m%d")
df['month'] = df.date.dt.month
df

NameError: name 'pd' is not defined

## Cleaning

In [4]:
df.replace(-999.0,np.nan, inplace=True)
for i in df.columns.drop('date'):
    df[i] = df[i].interpolate(method='linear')
df

NameError: name 'df' is not defined

## Scaling and Normalization

Standardization: Using the StandardScaler we will scale the following:
 - Temperature (T)
 - Wind Speed (WS)
 - Clearness Index (CS)
 - Soil Surface Moisture (GW)
 - Evapotranspiration Rate (EL)
 - Surface Pressure (PS)

In [5]:
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
standard_scaling_cols = ['T',
                        'WS',
                        'CS',
                        'GW',
                        'EL',
                        'PS']

df[standard_scaling_cols] = standard_scaler.fit_transform(df[standard_scaling_cols])
df


NameError: name 'df' is not defined

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmaxscaler = MinMaxScaler()
df['PRECIPITATIONCAL'] = minmaxscaler.fit_transform(df['PRECIPITATIONCAL'])

Cyclic Encoding:
- Apply Cyclic encoding to the WD parameter
- Drop WD

In [ ]:
df['WD_sin'] = np.sin(np.deg2rad(df.WD2M))
df['WD_cos'] = np.cos(np.deg2rad(df.WD2M))
df.drop(columns='WD', inplace=True)

MinMax Scaling:
Scale Features using that have a given range:
- Relative Humidity scaled down by 100 (since it is a percentage)
- Amplitude to be scaled down by 3 (0-3) is range of Amplitude

In [6]:
df.amplitude = df.amplitude/3
df.RH = df.RH/100
df

NameError: name 'df' is not defined

## EDA

In [7]:
df

NameError: name 'df' is not defined

In [8]:
sns.set_theme(style="darkgrid")
sns.lineplot(df.drop(columns='month'))

NameError: name 'sns' is not defined

## Create test Dataset

In [9]:
import json
import requests
import numpy as np
url =  "https://power.larc.nasa.gov/api/temporal/daily/point"

params = {
    "parameters":'T',
    "start": "20220101", #Get data from 2015 because we use standard scaler to scale, and test data is from 2015 to 2021
    "end": "20221231",
    "community": "RE",
    "latitude": str(19.67),
    "longitude": str(78.53),
    "user":"DAVEDownload",
    "format":"JSON",
    
}

response = requests.get(url, params=params)
json_data = json.loads(response.text)
json_data['properties']['parameter']
test_df = pd.DataFrame(json_data['properties']['parameter'])
# test_df.replace(-999.0, np.nan, inplace=True)
test_df['date'] = test_df.index
test_df['date'] = pd.to_datetime(test_df.date, format=r"%Y%m%d")
test_df.replace(-999.0,np.nan, inplace=True)
test_df['T2M'] = test_df['T2M'].interpolate(method='linear')
test_df.T2M.astype(np.float64)
# test_ss = StandardScaler()
# test_df['T2M'] = test_ss.fit_transform(test_df['T2M'].to_numpy().reshape(-1,1))
# test_df


test_df['month'] = test_df.date.dt.month
test_df

KeyError: 'properties'

# Testing Models (DARTS)

## Create Timesereies

In [ ]:
import pandas as pd
from darts import TimeSeries

series = TimeSeries.from_dataframe(df, "date", df.columns.drop('date'))

target = series['T']
train_series = series
test_series = TimeSeries.from_dataframe(test_df, "date", ["T2M",'month'])


past_covariates = series[['RH',
                          'WS',
                          'CS',
                          'GW',
                          'EL',
                          'PS',
                          'PRECIPITATIONCAL',
                          'MODEL',
                          'PC1',
                          'PC2',
                          'amplitude',
                          'month',
                          'WD_sin',
                          'WD_cos']]

future_covariates = series['month'].append(test_series['month'])



## NLinearModel


In [ ]:
from darts.models import  NLinearModel
import torch
Heatwave_NLinearModel =  NLinearModel(
                     model_name='Heatwave_NLinear',
                     loss_fn=torch.nn.HuberLoss(),
                     input_chunk_length=720,
                     output_chunk_length=365,
                     batch_size=32,
                     n_epochs=100,
                     add_encoders={
                        'cyclic': {'future': ['month'], 'past':['month']},
                        },
                     force_reset=True,
                     pl_trainer_kwargs={
                        "accelerator": "gpu",
                        "devices": [0]
                      },
                      log_tensorboard=True
                    )

## TCN Model: Temporal Convolutional Network

In [ ]:
from darts.models.forecasting.tcn_model import TCNModel
import torch
Heatwave_TCNModel = TCNModel(
                     model_name='TCN_Heatwave',
                     loss_fn=torch.nn.HuberLoss(),
                     input_chunk_length=1095,
                     output_chunk_length=365,
                     batch_size=256,
                     n_epochs=1000,
                     add_encoders={
                        'cyclic': {'past':['month']},
                        },
                     force_reset=True,
                     pl_trainer_kwargs={
                        "accelerator": "gpu",
                        "devices": [0]
                      },
                      log_tensorboard=True
                    )
# model.fit(train_series)


## TFT Model: Temporal Fusion Transformer

In [10]:
from darts.models.forecasting.tft_model import TFTModel
import torch
import torchmetrics

torch.cuda.empty_cache()
Heatwave_TFTModel = TFTModel(
                     model_name='Heatwave_TFT2',
                     input_chunk_length=365,
                     output_chunk_length=365,
                     batch_size=32,
                     # torch_metrics=regression_metrics,
                     full_attention=True,
                     feed_forward='ReGLU',
                     n_epochs=50,
                     add_encoders={
                        'cyclic': {'future': ['month'], 'past':['month']},
                        },
                     force_reset=True,
                     pl_trainer_kwargs={
                        "accelerator": "gpu",
                        "devices": [0]
                      },
                      log_tensorboard=True
                    )
# model.fit(train_series, verbose=True)


ModuleNotFoundError: No module named 'darts'

## Fit, Predict and Metrics

In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
def get_metrics(actual: TimeSeries, forecast: TimeSeries):
    
    actual = test_series['T'].values()
    forecast = forecast['T'].values()

    mae = mean_absolute_error(actual, forecast)
    mse = mean_squared_error(actual, forecast)
    rmse = np.sqrt(mse)
    r2 = r2_score(actual, forecast)

    print("Mean Absolute Error (MAE): {:.4f}".format(mae))
    print("Mean Squared Error (MSE): {:.4f}".format(mse))
    print("Root Mean Squared Error (RMSE): {:.4f}".format(rmse))
    print("R-squared (R2): {:.4f}".format(r2))

NameError: name 'TimeSeries' is not defined

In [12]:
import matplotlib.pyplot as plt
def fit_predict_plot_heatwave(model, predicition_horizon=365, plot_previous_years=False, is_prefitted=False):
    if not is_prefitted:
        model.fit(train_series, 
                    verbose=True,
                    past_covariates = past_covariates,
                    future_covariates = future_covariates 
                    )
    
    forecast = model.predict(  predicition_horizon,
                                past_covariates = past_covariates,
                                future_covariates = future_covariates
                                )
    
    #Inverse scale forecast values
    forecast = forecast['T'] * standard_scaler.scale_[0] + standard_scaler.mean_[0]
    # print(forecast)
    plt.figure(figsize=(20,10))
    if plot_previous_years:
        series[['T']].plot()
    test_series['T'].plot()
    forecast['T'].plot(label="forecast")


    plt.savefig(f'darts_logs/{model.model_name}/{model.model_name}_plot.png')
    plt.legend()
    plt.show()
    

    get_metrics(actual=test_series, forecast=forecast)
    
    
    

In [13]:
fit_predict_plot_heatwave(Heatwave_TCNModel)

NameError: name 'Heatwave_TCNModel' is not defined

In [ ]:
fit_predict_plot_heatwave(Heatwave_TFTModel)

In [ ]:
Heatwave_TCNModel.save('..\models\darts_logs\TCN_Heatwave\TCN_Heatwave.pt')

In [ ]:
Heatwave_TFTModel.save('..\models\darts_logs\Heatwave_TFT\TFT_Heatwave.pt')